# Molecular Descriptors using PadelPy

[@ Kushal Raj Roy](https://twitter.com/kushalroy59)

Beta lactamase is an enzyme produced by bacteria which produced multi-ressistance against antibiotics. In this notebook we will apply 10 moleular fingerprints to investigate  molecular activities. 

       
 'AtomPairs2DCount',
 'AtomPairs2D',
 'EState',
 'CDKextended',
 'CDK',
 'CDKgraphonly',
 'KlekotaRothCount',
 'KlekotaRoth',
 'MACCS',
 'PubChem',
 'SubstructureCount',
 'Substructure'
 
 
 Notebook Outlines: 
 
 1. Preparation of fingerprint present in XML file.
 2. Prepare dataset subset as an input for PadelPy.
 3. Machine Learning model building.
 4. Cross Validation.
 
 
 This is an open project proposed by:


[Chanin Nantasenamat](https://scholar.google.com/citations?user=df-l7zQAAAAJ&hl=en)

[Data Professor YouTube channel](https://youtube.com/dataprofessor)

### Let's Learn Together!


## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("betalactamase_bioactivity_data_3class_pIC50.csv")

In [3]:
df

,Unnamed: 0,molecule_chembl_id,canonical_smiles,class,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,0,CHEMBL3112752,N[C@@H](Cc1ccc(NC(=O)[C@@H]2CC[C@@H]3CN2C(=O)N...,inactive,428.423,-0.4175,4.0,7.0,4.552842
1,1,CHEMBL3112746,O=C(Nc1ccncc1)[C@@H]1CC[C@@H]2CN1C(=O)N2OS(=O)...,inactive,342.333,0.0231,2.0,6.0,4.698970
2,3,CHEMBL1172388,CCC(S)P(=O)(O)O,inactive,156.143,0.8300,3.0,2.0,4.823909
3,5,CHEMBL91287,CCCCc1nn(-c2ccc(Cl)cc2)c(C(=O)OCC)c1Cc1ccc(-c2...,inactive,541.055,6.4829,1.0,7.0,4.154902
4,6,CHEMBL93289,CCCCc1nn(-c2cccc(Cl)c2)c(C(=O)OCC)c1Cc1ccc(-c2...,inactive,541.055,6.4829,1.0,7.0,4.119186
...,...,...,...,...,...,...,...,...,...
44014,63005,CHEMBL858,O=C(O)CN(CCN(CC(=O)O)CC(=O)O)CC(=O)O,inactive,292.244,-2.0712,4.0,6.0,4.881002
44015,63007,CHEMBL86905,O=Cc1coc2ccccc2c1=O,inactive,174.155,1.6055,0.0,3.0,4.273028
44016,63010,CHEMBL87719,CC1(C)[C@H](C(=O)O)N2C(=O)[C@]3(C[C@@H]3OC3CCC...,active,357.428,0.9229,1.0,5.0,6.469274
44017,63012,CHEMBL9306,O=C([O-])[C@H]1/C(=C/CO)O[C@@H]2CC(=O)N21.[Li+],active,198.154,-2.4303,1.0,5.0,6.629377


## 1. Preparation of fingerprint present in XML file.

### 1.1 Preparation of fingerprint present in XML file. 

In [4]:
! wget https://github.com/dataprofessor/padel/raw/main/fingerprints_xml.zip
! unzip fingerprints_xml.zip y

--2021-12-12 22:04:45--  https://github.com/dataprofessor/padel/raw/main/fingerprints_xml.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dataprofessor/padel/main/fingerprints_xml.zip [following]
--2021-12-12 22:04:46--  https://raw.githubusercontent.com/dataprofessor/padel/main/fingerprints_xml.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10871 (11K) [application/zip]
Saving to: ‘fingerprints_xml.zip.20’

fingerprints_xml.zi 100%[===================>]  10.62K  --.-KB/s    in 0s      

2021-12-12 22:04:46 (63.2 MB/s) - ‘fingerprints_xml.zip.20’ saved [10871/10871]

Archive:  finge

In [ ]:
#import glob to read xml file
import glob
xml_files = glob.glob("*.xml")
xml_files.sort()
xml_files

### 1.2 create a fingerprint list 

In [6]:
FP_list = ['AtomPairs2DCount',
 'AtomPairs2D',
 'EState',
 'CDKextended',
 'CDK',
 'CDKgraphonly',
 'KlekotaRothCount',
 'KlekotaRoth',
 'MACCS',
 'PubChem',
 'SubstructureCount',
 'Substructure']

In [7]:
#create a dictionary
fp = dict(zip(FP_list, xml_files))
fp

{'AtomPairs2DCount': 'AtomPairs2DFingerprintCount.xml',
 'AtomPairs2D': 'AtomPairs2DFingerprinter.xml',
 'EState': 'EStateFingerprinter.xml',
 'CDKextended': 'ExtendedFingerprinter.xml',
 'CDK': 'Fingerprinter.xml',
 'CDKgraphonly': 'GraphOnlyFingerprinter.xml',
 'KlekotaRothCount': 'KlekotaRothFingerprintCount.xml',
 'KlekotaRoth': 'KlekotaRothFingerprinter.xml',
 'MACCS': 'MACCSFingerprinter.xml',
 'PubChem': 'PubchemFingerprinter.xml',
 'SubstructureCount': 'SubstructureFingerprintCount.xml',
 'Substructure': 'SubstructureFingerprinter.xml'}

### 1.3 import beta_lactamase_pIC50_csv file. 

N.B. find the csv file here > in this [Git-Hub_repository](https://github.com/kushalrajroy/beta_lactamase_project/blob/eeb82835d5abe236e3e0218a9df90f39cf9ab741/betalactamase_bioactivity_data_3class_pIC50.csv)

In [8]:
import pandas as pd

df = pd.read_csv("betalactamase_bioactivity_data_3class_pIC50.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44019 entries, 0 to 44018
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          44019 non-null  int64  
 1   molecule_chembl_id  44019 non-null  object 
 2   canonical_smiles    44019 non-null  object 
 3   class               44019 non-null  object 
 4   MW                  44019 non-null  float64
 5   LogP                44019 non-null  float64
 6   NumHDonors          44019 non-null  float64
 7   NumHAcceptors       44019 non-null  float64
 8   pIC50               44019 non-null  float64
dtypes: float64(5), int64(1), object(3)
memory usage: 3.0+ MB


## 2. Prepare dataset subset as an input for PadelPy

In [9]:
df2 = pd.concat( [df['canonical_smiles'],df['molecule_chembl_id']], axis=1 )
df2.to_csv('molecule.smi', sep='\t', index=False, header=False)
df2

,canonical_smiles,molecule_chembl_id
0,N[C@@H](Cc1ccc(NC(=O)[C@@H]2CC[C@@H]3CN2C(=O)N...,CHEMBL3112752
1,O=C(Nc1ccncc1)[C@@H]1CC[C@@H]2CN1C(=O)N2OS(=O)...,CHEMBL3112746
2,CCC(S)P(=O)(O)O,CHEMBL1172388
3,CCCCc1nn(-c2ccc(Cl)cc2)c(C(=O)OCC)c1Cc1ccc(-c2...,CHEMBL91287
4,CCCCc1nn(-c2cccc(Cl)c2)c(C(=O)OCC)c1Cc1ccc(-c2...,CHEMBL93289
...,...,...
44014,O=C(O)CN(CCN(CC(=O)O)CC(=O)O)CC(=O)O,CHEMBL858
44015,O=Cc1coc2ccccc2c1=O,CHEMBL86905
44016,CC1(C)[C@H](C(=O)O)N2C(=O)[C@]3(C[C@@H]3OC3CCC...,CHEMBL87719
44017,O=C([O-])[C@H]1/C(=C/CO)O[C@@H]2CC(=O)N21.[Li+],CHEMBL9306


### 2.1 Calculate descriptors

fingerprints(fp)

In [10]:
#lets look at fingerprints(fp)
fp

{'AtomPairs2DCount': 'AtomPairs2DFingerprintCount.xml',
 'AtomPairs2D': 'AtomPairs2DFingerprinter.xml',
 'EState': 'EStateFingerprinter.xml',
 'CDKextended': 'ExtendedFingerprinter.xml',
 'CDK': 'Fingerprinter.xml',
 'CDKgraphonly': 'GraphOnlyFingerprinter.xml',
 'KlekotaRothCount': 'KlekotaRothFingerprintCount.xml',
 'KlekotaRoth': 'KlekotaRothFingerprinter.xml',
 'MACCS': 'MACCSFingerprinter.xml',
 'PubChem': 'PubchemFingerprinter.xml',
 'SubstructureCount': 'SubstructureFingerprintCount.xml',
 'Substructure': 'SubstructureFingerprinter.xml'}

### 2.2 import padel-descriptor

In [11]:
from padelpy import padeldescriptor

fingerprint = 'Substructure'

fingerprint_output_file = ''.join([fingerprint,'.csv']) #Substructure.csv
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='molecule.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

### 2.3 Display calculated fingerprints

In [12]:
descriptors = pd.read_csv(fingerprint_output_file)
descriptors

,Name,SubFP1,SubFP2,SubFP3,SubFP4,SubFP5,SubFP6,SubFP7,SubFP8,SubFP9,...,SubFP298,SubFP299,SubFP300,SubFP301,SubFP302,SubFP303,SubFP304,SubFP305,SubFP306,SubFP307
0,CHEMBL3112746,0,1,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
1,CHEMBL3112752,0,1,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
2,CHEMBL1172388,1,1,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
3,CHEMBL91287,1,1,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
4,CHEMBL93289,1,1,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44014,CHEMBL86905,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,1,0,0,0,1
44015,CHEMBL85799,1,1,1,1,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1
44016,CHEMBL9306,0,1,0,0,0,0,0,0,0,...,0,1,1,1,1,0,0,0,0,1
44017,CHEMBL87719,1,1,0,1,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1


In [19]:
# Build a Random Forest Model

X = descriptors.drop('Name', axis=1)
y = df['class']

In [20]:
##Remove low variance features

In [21]:
from sklearn.feature_selection import VarianceThreshold

def remove_low_variance(input_data, threshold=0.1):
    selection = VarianceThreshold(threshold)
    selection.fit(input_data)
    return input_data[input_data.columns[selection.get_support(indices=True)]]

X = remove_low_variance(X, threshold=0.1)
X

,SubFP1,SubFP2,SubFP3,SubFP18,SubFP23,SubFP26,SubFP85,SubFP88,SubFP96,SubFP100,...,SubFP137,SubFP143,SubFP171,SubFP180,SubFP181,SubFP182,SubFP183,SubFP184,SubFP214,SubFP287
0,0,1,0,0,0,0,0,1,0,1,...,0,1,0,0,1,0,0,1,0,0
1,0,1,0,0,0,0,0,1,0,1,...,0,1,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,1,1,0,0,...,1,0,1,1,1,0,0,1,0,1
4,1,1,0,0,0,0,1,1,0,0,...,1,0,1,1,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44014,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1
44015,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44016,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
44017,1,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


## 3. Machine Learning model building

In [22]:
#Data splitting

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#model building

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import matthews_corrcoef

model = RandomForestClassifier(n_estimators=500, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=500, random_state=42)

### 3.1 Apply model to make prediction

In [23]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

### 3.2 Calculate model performance metrics

In [24]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)


In [25]:
mcc_train = matthews_corrcoef(y_train, y_train_pred)
mcc_train

0.4268793837622389

In [29]:
mcc_test = matthews_corrcoef(y_test, y_test_pred)
mcc_test

0.014787579894008435

## 4. Cross-validation

In [30]:
from sklearn.model_selection import cross_val_score

rf = RandomForestClassifier(n_estimators=500, random_state=42)
cv_scores = cross_val_score(rf, X_train, y_train, cv=5)
cv_scores

array([0.66349567, 0.66590941, 0.66661934, 0.66747125, 0.66576743])

In [31]:
mcc_cv = cv_scores.mean()
mcc_cv

0.6658526196223201

In [32]:
model_name = pd.Series(['Random forest'], name='Name')
mcc_train_series = pd.Series(mcc_train, name='MCC_train')
mcc_cv_series = pd.Series(mcc_cv, name='MCC_cv')
mcc_test_series = pd.Series(mcc_test, name='MCC_test')

performance_metrics = pd.concat([model_name, mcc_train_series, mcc_cv_series, mcc_test_series], axis=1)
performance_metrics

,Name,MCC_train,MCC_cv,MCC_test
0,Random forest,0.426879,0.665853,0.014788
